In [1]:
import numpy as np
import uproot
import datetime as dt
from sndaq.analysis import AnalysisHandler, AnalysisConfig
from sndaq.trigger import TriggerHandler
from sndaq.util.legacy import get_sndata_file
from pathlib import Path
import os

## Setup Analyses and Storage

In [2]:
def nstimes_to_hrtimes(ns_time, year=None):
    """Convert ns since start of year times to human readable (hr) times
    
    ns_time : numpy.array
        Array of times, measured since 01/01/`year` 00:00:00.000 with nanosecond precision
    year : int
        Year corresponding to ns times
        
    Returns
    -------
    dt_start : np.datetime64
        datetime corresponding to start of ns_times (run_start)
    nt_time_norm : numpy.array
        Time bins relative to dt_start in units s, should be uniform 0.5s
    """
    if year is None:
        year = dt.datetime.now().year
    dt_start = np.datetime64(f'{year}-01-01') + np.timedelta64(ns_time[0], 'ns')
    ns_time_norm = (ns_time - ns_time[0]) / 1e9      
    return dt_start, ns_time_norm


def get_500ms_rundata(file, year=None):
    """Get 500ms rundata from sndaq .root file
    
    Parameters
    ----------
    file : string or Pathlike
        Path to SNDAQ data file
    year : int
        Year of run during which data was collected
        
    Returns
    -------
    data : np.ndarray with Dim(n_500ms_bins, n_doms)
        500ms data for all doms
    start : np.datetime64
        Datetime corresponding to start of data 
    time_bins : numpy.array
        Time bins relative to start in units s, should be uniform 0.5s
    """
    if ''.join(Path(file).suffixes) == '.tar.gz':
        tarball_path = Path(file)
        parent_dir = tarball_path.parent
        sndata_rootfile = str(tarball_path).rstrip(''.join(tarball_path.suffixes))+'.root'
        
        if os.path.exists(sndata_rootfile):
            print(f'Provided path to {file} \n\tbut found extracted {os.path.basename(sndata_rootfile)} '+
                  'in the same directory. Switching to ROOT file.')
            
        else:
            if os.path.exists(file):
                print(f'Extracting ROOT file {os.path.basename(rootfile)} from file: {file}')
                with tarfile.open(file) as tf:
                    tf.extract(os.path.basename(rootfile), path=parent_dir)
            else:
                raise FileNotFoundError(f"No SN Data file {file} found!")
        file = sndata_rootfile        
    
    with uproot.open(file) as f:
        sn_all = f['sn_all/sn_all_data']
        data = sn_all['data'].array().to_numpy()  # Might hang a little
        gps_time_high = sn_all['SN_Streaming_t/gps_time_high'].array().to_numpy() # First 32 bits of times
        gps_time_low = sn_all['SN_Streaming_t/gps_time_low'].array().to_numpy()  # Last 32 bits of times
        gps_time = np.bitwise_or(np.left_shift(gps_time_high, 32), gps_time_low)
    start, time = nstimes_to_hrtimes(gps_time, year)
    
    return data, start, time

In [3]:
sndata_file = get_sndata_file(137104, USER_live='icecube', PASS_live='skua', USER_ldap='sgriswold',
                              cache_dir = '../../../scratch/data/rundata/SPS/')[0]
data, start, time = get_500ms_rundata(sndata_file)

idc_dropped = np.where(np.all(data==0, axis=0))[0]
data = np.delete(data, idc_dropped, axis=1)
n_dropped_doms = idc_dropped.size

print(f"\nLoaded {data.shape[0] * 0.5} s of data ({data.shape[0]} bins) for {data.shape[1]} DOMs")

Found SN Data file(s): ../../../scratch/data/rundata/SPS/sndata_137104_000.tar.gz
Provided path to ../../../scratch/data/rundata/SPS/sndata_137104_000.tar.gz 
	but found extracted sndata_137104_000.root in the same directory. Switching to ROOT file.

Loaded 28804.0 s of data (57608 bins) for 5075 DOMs


In [4]:
ndom = 5160 - n_dropped_doms
ana_conf = AnalysisConfig.from_config(conf_path='../../data/config/analysis.config')
ana = AnalysisHandler(config=ana_conf, dropped_doms=idc_dropped)

nbins_window = 400  # 200 s of rates in 0.5s bins
nbins_trigger = int((ana.config.duration_nosearch + 3*max(ana._binnings)) / 500 - 1)
nbins_fill_trailing = nbins_trigger - int(ana.config.duration_nosearch/2 /500)
nbins_fill_leading = nbins_trigger - nbins_fill_trailing - nbins_window

dom_bkg_mu = 140
dom_bkg_sig = 22

rate_500ms = np.zeros(shape=(nbins_window, ndom), dtype=np.uint16)
signi = np.zeros(shape=(nbins_window, len(ana.analyses)), dtype=float)

filler = None
np.random.seed(42)

## Feeding 500 ms data to PySNDAQ

In [5]:
# Define a fresh instance for convenience of repeatedly running this cell
ana = AnalysisHandler(config=ana_conf, dropped_doms=idc_dropped)
alert = TriggerHandler()

for idx_time, rate in enumerate(data):

    ana.buffer_analysis.append(rate.astype(np.uint32))
    ana.update_analyses()
    ana.process_triggers()

    if ana.trigger_finalized:
        ana.cand_count += 1
        alert.process_trigger(ana.current_trigger)
        ana.current_trigger.reset()
        ana.trigger_count = 0
        print('')

Primary Trigger Processing...
Candidate   1, Trigger   1: xi=5.003 in   0.5 +(0.0) s Analysis @t=123.0 s

Primary Trigger Processing...
Candidate   2, Trigger   1: xi=4.622 in   0.5 +(0.0) s Analysis @t=311.0 s

Primary Trigger Processing...
Candidate   3, Trigger   1: xi=5.091 in   0.5 +(0.0) s Analysis @t=344.0 s

Primary Trigger Processing...
Candidate   4, Trigger   3: xi=4.603 in   1.5 +(0.0) s Analysis @t=616.5 s

Primary Trigger Processing...
Candidate   5, Trigger   1: xi=4.116 in   1.5 +(0.0) s Analysis @t=928.5 s

Primary Trigger Processing...
Candidate   6, Trigger   1: xi=4.022 in   1.5 +(0.0) s Analysis @t=963.0 s

Primary Trigger Processing...
Candidate   7, Trigger   1: xi=4.367 in   1.5 +(1.0) s Analysis @t=1013.0 s

Primary Trigger Processing...
Candidate   8, Trigger   1: xi=4.378 in   1.5 +(0.0) s Analysis @t=1086.0 s

Primary Trigger Processing...
Candidate   9, Trigger   1: xi=5.175 in   0.5 +(0.0) s Analysis @t=1486.0 s

Primary Trigger Processing...
Candidate  10

Primary Trigger Processing...
Candidate  78, Trigger   2: xi=5.116 in   1.5 +(0.0) s Analysis @t=10444.5 s

Primary Trigger Processing...
Candidate  79, Trigger   2: xi=5.362 in   0.5 +(0.0) s Analysis @t=10564.0 s

Primary Trigger Processing...
Candidate  80, Trigger   1: xi=4.237 in   0.5 +(0.0) s Analysis @t=10635.0 s

Primary Trigger Processing...
Candidate  81, Trigger   1: xi=4.304 in   0.5 +(0.0) s Analysis @t=10740.5 s

Primary Trigger Processing...
Candidate  82, Trigger   1: xi=4.027 in   1.5 +(1.0) s Analysis @t=10788.5 s

Primary Trigger Processing...
Candidate  83, Trigger   1: xi=4.650 in   1.5 +(0.0) s Analysis @t=10993.5 s

Primary Trigger Processing...
Candidate  84, Trigger   2: xi=4.902 in   1.5 +(0.0) s Analysis @t=11364.0 s

Primary Trigger Processing...
Candidate  85, Trigger   1: xi=4.369 in   0.5 +(0.0) s Analysis @t=11626.0 s

Primary Trigger Processing...
Candidate  86, Trigger   1: xi=4.368 in   4.0 +(0.5) s Analysis @t=11717.5 s

Primary Trigger Processing..

Primary Trigger Processing...
Candidate 153, Trigger   2: xi=5.443 in   1.5 +(0.0) s Analysis @t=21580.5 s

Primary Trigger Processing...
Candidate 154, Trigger   1: xi=4.257 in   0.5 +(0.0) s Analysis @t=21855.0 s

Primary Trigger Processing...
Candidate 155, Trigger   3: xi=4.779 in   0.5 +(0.0) s Analysis @t=21991.5 s

Primary Trigger Processing...
Candidate 156, Trigger   1: xi=4.014 in   0.5 +(0.0) s Analysis @t=22095.5 s

Primary Trigger Processing...
Candidate 157, Trigger   4: xi=4.560 in   1.5 +(0.5) s Analysis @t=22226.5 s

Primary Trigger Processing...
Candidate 158, Trigger   1: xi=4.576 in  10.0 +(4.5) s Analysis @t=22275.5 s

Primary Trigger Processing...
Candidate 159, Trigger   1: xi=4.037 in   0.5 +(0.0) s Analysis @t=22409.0 s

Primary Trigger Processing...
Candidate 160, Trigger   1: xi=5.028 in   1.5 +(0.0) s Analysis @t=22482.0 s

Primary Trigger Processing...
Candidate 161, Trigger   1: xi=4.050 in   0.5 +(0.0) s Analysis @t=22818.5 s

Primary Trigger Processing..

In [6]:
for i, search in enumerate(ana.analyses):
    idc_debug = {
        'eod': search.idx_eod,
        'bgl': search.idx_bgl,
        'exl': search.idx_exl,
        'sw':  search.idx_sw,
        'ext': search.idx_ext,
        'bgt': search.idx_bgt
    }
    
    print(f"\nSearch #{i:<2d} :{search.binsize/1e3:4.1f} +{search.offset/1e3:<4.1f}s")
    for key, val in idc_debug.items():
        print(f"{key:>10s} : {val:<5d}")


Search #0  : 0.5 +0.0 s
       eod : 1319 
       bgl : 719  
       exl : 689  
        sw : 688  
       ext : 658  
       bgt : 58   

Search #1  : 1.5 +0.0 s
       eod : 1319 
       bgl : 719  
       exl : 689  
        sw : 686  
       ext : 656  
       bgt : 56   

Search #2  : 1.5 +0.5 s
       eod : 1318 
       bgl : 718  
       exl : 688  
        sw : 685  
       ext : 655  
       bgt : 55   

Search #3  : 1.5 +1.0 s
       eod : 1317 
       bgl : 717  
       exl : 687  
        sw : 684  
       ext : 654  
       bgt : 54   

Search #4  : 4.0 +0.0 s
       eod : 1319 
       bgl : 719  
       exl : 689  
        sw : 681  
       ext : 651  
       bgt : 51   

Search #5  : 4.0 +0.5 s
       eod : 1318 
       bgl : 718  
       exl : 688  
        sw : 680  
       ext : 650  
       bgt : 50   

Search #6  : 4.0 +1.0 s
       eod : 1317 
       bgl : 717  
       exl : 687  
        sw : 679  
       ext : 649  
       bgt : 49   

Search #7  : 4.0 +1.5 s
  

In [7]:
for i, search in enumerate(ana.analyses):
    idc_debug = {
        'add_bgl': search._idx_addbgl[[0,-1]],
        'sub_bgl': search._idx_subbgl[[0,-1]],
        'add_sw': search._idx_addsw[[0,-1]],
        'sub_sw': search._idx_subsw[[0,-1]],
        'add_bgt': search._idx_addbgt[[0,-1]],
        'sub_bgt':  search._idx_subbgt[[0,-1]],
    }
    
    print(f"\nSearch #{i:<2d} :{search.binsize/1e3:4.1f} +{search.offset/1e3:<4.1f}s")
    for key, val in idc_debug.items():
        print(f"{key:>10s} : {val[0]:>5d} .. {val[1]:<5d}")


Search #0  : 0.5 +0.0 s
   add_bgl :  1318 .. 1318 
   sub_bgl :   718 .. 718  
    add_sw :   688 .. 688  
    sub_sw :   687 .. 687  
   add_bgt :   657 .. 657  
   sub_bgt :    57 .. 57   

Search #1  : 1.5 +0.0 s
   add_bgl :  1316 .. 1318 
   sub_bgl :   716 .. 718  
    add_sw :   686 .. 688  
    sub_sw :   683 .. 685  
   add_bgt :   653 .. 655  
   sub_bgt :    53 .. 55   

Search #2  : 1.5 +0.5 s
   add_bgl :  1315 .. 1317 
   sub_bgl :   715 .. 717  
    add_sw :   685 .. 687  
    sub_sw :   682 .. 684  
   add_bgt :   652 .. 654  
   sub_bgt :    52 .. 54   

Search #3  : 1.5 +1.0 s
   add_bgl :  1314 .. 1316 
   sub_bgl :   714 .. 716  
    add_sw :   684 .. 686  
    sub_sw :   681 .. 683  
   add_bgt :   651 .. 653  
   sub_bgt :    51 .. 53   

Search #4  : 4.0 +0.0 s
   add_bgl :  1311 .. 1318 
   sub_bgl :   711 .. 718  
    add_sw :   681 .. 688  
    sub_sw :   673 .. 680  
   add_bgt :   643 .. 650  
   sub_bgt :    43 .. 50   

Search #5  : 4.0 +0.5 s
   add_bgl